In [220]:
import numpy as np

class scalar:
    
    def __init__(self, data, label=None, parents=None, same_level=None):
        self.data = data
        self.gradient = 1.0
        self.label = label
        self.parents = parents
        self.same_level = None
        
    def __add__(self, other):
        other = other if isinstance(other, scalar) else scalar(other, str(other), None)
        other.same_level = self
        self.same_level = other
        out = scalar(self.data + other.data, f"{self.label}+{other.label}", parents=[self, '+', other]) 
        
        return out
    
    def __mul__(self, other):
        other = other if isinstance(other, scalar) else scalar(other, str(other), None)
        other.same_level = self
        self.same_level = other
        out = scalar(self.data*other.data, self.label + other.label, parents=[self, '*', other])        
        
        return out
    
    def __pow__(self, other):
        other = other if isinstance(other, scalar) else scalar(other, str(other), None)
        other.same_level = self
        self.same_level = other
        out = scalar(self.data**other.data, f"{self.label}^{other.label}", parents=[self, '**', other]) 
        
        return out
    
    def __repr__(self):
        return f"{self.data, self.gradient, self.label, [p.label if type(p) != str else p for p in self.parents] if self.parents != None else self.parents, self.same_level.label if self.same_level != None else self.same_level}"
        
    def back_propagate(self):
        if self.parents == None:
            return 
        
        elif self.parents[1] == '*':
            self.parents[0].gradient *= self.gradient * self.parents[2].data
            self.parents[2].gradient *= self.gradient * self.parents[0].data
            
            self.parents[0].back_propagate()
            self.parents[2].back_propagate()
            
        elif self.parents[1] == '**':
            self.parents[0].gradient *= self.gradient * self.parents[2].data * self.parents[0].data ** (self.parents[2].data-1)
            self.parents[2].gradient *= self.gradient * self.data * np.log(self.parents[0].data)
            
            self.parents[0].back_propagate()
            self.parents[2].back_propagate()
        
        elif self.parents[1] == '+':
            self.parents[0].gradient *= self.gradient
            self.parents[2].gradient *= self.gradient
            
            self.parents[0].back_propagate()
            self.parents[2].back_propagate()
            
    def wash(self):
        if self.parents == None:
            return
        
        print(self.get_parameters())
        
        self.parents[0].gradient = 1.0
        self.parents[0].wash()
        
        self.parents[2].gradient = 1.0
        self.parents[2].wash()
        
    def __neg__(self): 
        return self * -1

    def __radd__(self, other): 
        return self + other

    def __sub__(self, other): 
        return self + (-other)

    def __rsub__(self, other):
        return other + (-self)

    def __rmul__(self, other):
        return self * other

    def __truediv__(self, other):
        return self * other**-1

    def __rtruediv__(self, other):
        return other * self**-1
            

In [225]:
a = scalar(3, 'a')
b = scalar(2, 'b')
c = scalar(3, 'c')

In [226]:
d = a**b

In [227]:
d.back_propagate()

In [229]:
b

(2, 9.887510598012987, 'b', None, 'a')

In [231]:
9 * np.log(3)

9.887510598012987